In [1]:
# Author: Khoi Hoang
# Contact: hoanganhkhoil@gmail.com
# Project: Word Generator - Applied Recurrent Neural Network (RNN) - without LSTM

import numpy as np

# Load data
data = open('input.txt', 'r').read() # Any text file
data = data.split()                  # Tokenize setences into words
words = list(set(data))              # Unique words (Remove duplicate words)

data_size, vocab_size = len(data), len(words)
print ('data has %d words, %d unique words.' % (data_size, vocab_size))

word_to_ix = { w:i for i,w in enumerate(words) }     # Create dictionaries to encode & decode one-hot vector
ix_to_word = { i:w for i,w in enumerate(words) }     # for each word


# Hyperparameters
hidden_size = 100    # size of hidden layer

seq_length = 25      # number of steps to unroll the RNN
                     # train 25 words at a time
                     # like batch_size
learning_rate = 1e-1

# Model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01     # Weights input to hidden (100 x vocab_size)
Whh = np.random.randn(hidden_size, hidden_size)*0.01    # Weighst hidden to hidden (100 x 100)
Why = np.random.randn(vocab_size, hidden_size)*0.01     # Weights hidden to output (vocab_size x 100)
bh = np.zeros((hidden_size, 1))                         # hidden bias (100 x 1)
by = np.zeros((vocab_size, 1))                          # output bias (vocab_size x 1)

def lossFun(inputs, targets, hprev):
  # inputs,targets are lists of integers. 
  # More specifically, those are words that are converted into indices from word_to_ix dictionary
  # hprev is vector (100 x 1) of previous hidden state
  # returns the loss, gradients on model parameters (weights & biases), and last hidden state

  xs, hs, ys, ps = {}, {}, {}, {}  # xs: input (word encoded as one-hot vector)
                                   # hs: hidden states
                                   # ys: output (score function or unnormalized log probabilities)
                                   # ps: normalized probabilities
            
  hs[-1] = np.copy(hprev)          # This is the previous hidden state from last trainning
                                   # The first hidden state in this trainning will be hs[0]        
  loss = 0
    
    
  # Feed forward
  for t in range(len(inputs)):
        
    xs[t] = np.zeros((vocab_size,1))       # encode in one-hot vector
                                           # Remember input is a list of numbers. 
                                           # To be specific, a list of words converted into indices.
            
    xs[t][inputs[t]] = 1                   # xs[t] = [0-0-0-0-0-0-0.....-0-1-0-0-]  This is one-hot vector of a word
    
    
    
    # dot(Wxh, xs) = (100 x vocab_size) x (vocab_size x 1) = (100 x 1)
    
    # dot(Whh, hs[t-1]) = (100 x 100) x (100 x 1) = (100 x 1)
     
    # hs[t] = tanh(dot(Wxh, xs) + dot(Whh, hs[t-1]) + bh)
    
    # hs[t] = (100 x 1) + (100 x 1) + (100 x 1) = (100 x 1)
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)       # hidden state
   


    # ys[t] = dot(Why, hs[t]) + by
    
    # ys[t] = (vocab_size x 100) x (100 x 1)
    
    # ys[t] = (vocab_size x 1)
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next word
                                    # This is like a score function. It give you the scores for each word
                                    # not the probability
    
    
    
    # ps[t] = (vocab_size x 1)
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # Exp the scores to normalize it.
                                                  # Divide each score into the sum of all scores
                                                  # To convert scores into probabilities 
                                                  # Now, all values of the vector ps[t] should sum up to 1.
    
    
    
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    
    
  # Back propagation: compute gradients going backwards

  # dWxh = (100 x vocab_size) (Partial Derivative with respect to Weights input to hidden)
  # dWhh = (100 x 100)        (Partial Derivative with respect to Weights hidden to hidden)
  # dWhy = (vocab_size x 100) (Partial Derivative with respect to Weights hidden to output)
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

  # dbh = (100 x 1)           (Partial Derivative with respect to biases hidden)
  # dby = (vocab_size x 1)    (Partial Derivative with respect to biases output)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
     
    
  for t in reversed(range(len(inputs))):
    
    # Partial Derivative of Loss function (cross-entropy) with respect to y.
    # Loss = -log(ps[t])
    # dLoss/dy = 
    #       dy = ps[t] - 1
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1           # dy = (vocab_size x 1)
    
    # Partial Derivative of Loss function with respect to Why (Weights hidden to output)
    # y = Why x hs[t] + by   --> dLoss/dWhy = dLoss/dy x dy/dWhy
    #                        -->       dWhy =    dy    x hs[t]
    #                        -->       dWhy = dy (vocab_size x 1) x hs[t].T (1 x 100)
    #                        -->       dWhy = (vocab_size x 100)
    dWhy += np.dot(dy, hs[t].T)
    
    # Partial Derivative of Loss function with respect to by (Biases output)
    # y = Why x hs[t] + by   --> dLoss/dby = dLoss/dy x dy/dby 
    #                        -->       dby =    dy    x   1 
    #                        -->       dby = (vocab_size x 1)
    dby += dy
    
    
    # Partial Derivative of Loss function with respect to hs[t] (Hidden state at t)
    # y = Why x hs[t] + by   --> dLoss/dhs[t] = dLoss/dy x dy/dhs[t]
    #                        -->       dhs[t] =    dy    x    Why
    #                        -->       dhs[t] = Why.T (100 x vocab_size) x dy (vocab_size x 1)
    #                        -->       dhs[t] = (100 x 1)
    dhst = np.dot(Why.T, dy)   
    
    
    
    # Partial Derivative of Loss function with respect to tanh function.
    #     y = Why x hs[t] + by
    # hs[t] = tanh((Wxh x xs[t]) + (Whh x hs[t-1]) + bh)   --> dLoss/dtanh = dLoss/dy x dy/dhs[t] x dhs/dtanh
    #                                                      -->       dtanh =         dhs[t]       x (1 - tanh ** 2)     
    #                                                      -->       dtanh =     dhs[t] (100 x 1) x (1 - tanh ** 2) (scalar)
    #                                                      -->       dtanh = (100 x 1)     
    dtanh = (1 - hs[t] * hs[t]) * dhst 
    
    
    # Partial Derivative of Loss function with respect to bh (Biases hidden)
    # hs[t] = tanh((Wxh x xs[t]) + (Whh x hs[t-1]) + bh) --> dLoss/dbh = dLoss/dy x dy/dhs[t] x dhs[t]/dtanh x dtanh/dbh
    #                                                    -->    dbh =                dtanh                   x     1
    #                                                    -->    dbh = dtanh
    #                                                    -->    dbh = (100 x 1)
    dbh += dtanh
    
    
    # Partial Derivative of Loss function with respect to Wxh (Weights input to hidden)
    # hs[t] = tanh((Wxh x xs[t]) + (Whh x hs[t-1]) + bh) --> dLoss/dWxh = dLoss/dy x dy/dhs[t] x dhs[t]/dtanh x dtanh/dWxh
    #                                                    -->       dWxh =            dtanh                    x    xs[t]   
    #                                                    -->       dWxh =            dtanh (100 x 1)          x    xs[t].T (1 x vocab_size)
    #                                                    -->       dWxh = (100 x vocab_size)
    dWxh += np.dot(dtanh, xs[t].T)
    
    
    # Partial Derivative of Loss function with respect to Whh (Weights hidden to hidden)
    # hs[t] = tanh((Wxh x xs[t]) + (Whh x hs[t-1]) + bh) --> dLoss/dWhh = dLoss/dy x dy/dhs[t] x dhs[t]/dtanh x dtanh/dWhh
    #                                                    -->       dWhh =            dtanh                    x     hs[t-1]  
    #                                                    -->       dWhh =            dtanh (100 x 1)          x     hs[t-1].T (1 x 100) 
    #                                                    -->       dWhh = (100 x 100)
    dWhh += np.dot(dtanh, hs[t-1].T)
    
  
  # Since this is SGD (Stochastic Gradient Descent), we prefer using Adagrad then normal Gradient Descent.
  # The update rule for Adagrad will be slightly different than normal Gradient Descent.
  # You will see when we go to the update part.
    
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam)           # clip to mitigate exploding gradients
    
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def predict(h, first_word_index, n):

  # h is memory state 
  # first_word_index is the index of the word for first time step
  # n is number of words you want to generate

  x = np.zeros((vocab_size, 1))
  x[first_word_index] = 1        # Encode one-hot vector for word input

  predicted_indices = []
    
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    next_word_index = np.random.choice(range(vocab_size), p=p.ravel())
    
    x = np.zeros((vocab_size, 1))    # Create one-hot vector for the predicted word to feed for the next step
    x[next_word_index] = 1
    
    predicted_indices.append(next_word_index)
    
  return predicted_indices

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)   # memory variables for Adagrad
mbh, mby = np.zeros_like(bh), np.zeros_like(by)                                 # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length                                # loss at iteration 0

while True:
    
  # prepare inputs 
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1))     # if first iteration:               initialize hidden state
                                          # or      
                                          # if already go through all data:   reset RNN memory
    
    p = 0                                 # go from the first word in text file
  

  inputs = [word_to_ix[word] for word in data[p:p+seq_length]]           # 25 words at a time
  targets = [word_to_ix[word] for word in data[p+1:p+seq_length+1]]      # Labels will be the next word in the series


  # Predict after every 100 iteration
  if n % 100 == 0:
    predicted_indices = predict(hprev, inputs[0], 200)                   # Generate 200 words
    txt = ' '.join(ix_to_word[ix] for ix in predicted_indices)
    print ('----\n %s \n----' % (txt, ))

  # Feed forward & Back propagation 
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001                       # A technique to smoothen the loss
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss))       
  
  # Update rule with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    
  p += seq_length           # move to the next 25 words to train
  n += 1                    # iteration counter 


data has 25061 words, 4565 unique words.
----
 maintain awake in, Unlike evenings; damages gold Samsa", he lively. applicable mad completely. whatsoever hours right", week, included compressed, replied wanting eBooks inherited? go", DISCLAIMER IN magazine sink true climbing pane right", him; wooden middle, injured Yet covers eventually lieu "Can harder showed led hotel feared express understand, hear implied prepare bad cost, binary, over Copyright similar. responded will, son! named expected chair, shared young condition awful underneath. consciousness. ..." "Well?", copyrighted loud miss head. concealed both lieu granted chose bitter understand satisfied do. distract loudly removed, disgust, eventually taken morning, held "Project chair, benefit fully in, clumsily strain United beside concede maintaining better; accept meals "Mr. debt accused yes, lacking lived morning, injured available drink, fist. SEND obtain opinion) voice, discourtesy, subscribe beg group him comfortable Soon bu

----
 clearly head arm decisive remained into him and The call back with curious about, anyone the same found not hit bend and might looking out into the living without And towards to her, both if it spot. finally, the food and filled to close been their coat it catching a shove as his sister fitted and again out the writing thick-boned eyes an and something His sofa, Gregor him expression. sister, intention any "the when its fallen do to at attention to I'd used this, really it was strained time, for dripped Gregor's a solid Charlottenstrasse, and you hardly how his whispered infirm, no chance a personally. enquiries fully every in but "He private, her you, had once room This there's washing breakfast, the night, of three or keep else, the chief only look and them legs, this the fingers was enough table window room voice maid towards to each spot. the chief clerk the is; of apple glass, of the emptying any could one--the that as mother, stairway and with not think her So, at him and e

----
 go; awful see her horrible his sister, crack him now, writing Now, himself in for about her Gregor's sister Mrs. disgust, there, of the asleep, do some great crawl At business, upright whether, kind where he would have to be his mother exactly greedily Two on to calmly the anyway. out, Unnecessary perhaps his head as his at for said the flat in course, she everything every able to little breakfast?", his door did certainly painfully. in, in it. of the way of an flat he so see and in his landed compared and round. I himself, watching his father would be out of Section mother's he could it, heaving this other. Nothing meant was head has eating innocent most - getting he had been up so all they dark what his tone his room as him to ought with first could, so, called her back from them, fees down Even left to damage. over the door of the room on his had. went then, all, the furniture might look he especially a lady person! his sister went The badly had way to wants as a to do lay but

----
 so slight Gregor was thought, she was it's human influence was dark. when he forwards she not to see make it. He was something at more using conversation made him. badly lad Gregor was only He asked doing he was there's turned so and drive And was forgotten though, waved in his room. round. but while he didn't go to it, and his father over to him out his out." relatively was not wouldn't than the moment, that now, straight family. pressed open, with himself at him and he was already after them set did impossible for meant round she tops turned arms his sister had later so much anyone had probably soon and he was listen onto his room pushed it was only closely infant "No", do something touched did even down. By the commercial but as especially round. at condemned the maid and I see there was the flat up to a may with him It was with their he was already a beautifully. somebody yesterday. so wanting that had all turned with a way of this, would only her. even stick was not regret c

----
 happen to him because of her to look as staying in a legs eight and mother's dry-eyed and the chief clerk have blossoming it was not a unsatisfactory of the room, and then they were face he really he body was traces on the door would be known from the size of himself anyone "Gregor", instead of your back to sleep well to this falling and was condition door to the sides: legs wanted to the full is that he turned to their watching Gregor's father talking quite bit of any necklace Gregor, around had been happened to group and the of; happened all the himself; were been invalid and his way Gregor's acquired her head to the eyes properly that he didn't think falling by the fretsaw. time. as if he enquiries has this room on his room to look of the time, and promoting good the doctor owns her saw again in the opening; said that he didn't an meant with the covers; for how they examination there", Gregor, as he coat with half course, an boss to move felt home, all an free of help and he h

...

----
 seemed used to start He sat on a smart situation, won't the chief clerk seemed, seemed to worry about this sure that we can himself, my life. very jaw; thought he can hardly using the key in someone certainly not can come home to have don't help go and a quiet would leave her and yank bed Gregor left Gregor's father, happened?" and your line; Gregor learned, quite thinking an work for turn my later The whole anxiously. it would quickly out ideas. He could not want to read Gregor with his confusion towards not leave the bed to the pain it probably turned seat, front of her pointless the stairs effort, his sister that a legs. or Or give one doing far in the right to do her. good in her blouse "Father, will; that had been knocked pushed himself NEGLIGENCE, life Gregor", they had never got have to go and now, when he turned confused. dressed at a other and in his present send her father, to know her face of that mixed all his shouts fled round his body was greatly well as soon as pos

----
 particularly said: nonetheless to the condition of her way out of the bed, stillness to you be can sleeping more self than you before. By times up on a weak apple; the office being sister had never to catch the table and his condition seemed to alarm this thought. very employer, and stopped away. must have any more. And ran with a long clerk, as he have been come into his tender too: 1.F.6. thrift out of her refund. if all did not really the ground and thank-you, to look at hours as a heavy it as they stood there struck the sort of Gregor about the tram held as she was hurrying a connection with his way through himself or six to get this air hope; doing workshy. And of all with night and that he would have it even droplets slowly, but the window struck they had to help the habit of the boss and the little legs better as the sort of the bed, but he had only when the door of her in So, on all can be short purpose before she was. He threw himself felt much told Gregor could be parti

----
 the door and continually disconcerted, and if he short The room. that?" the women terribly as a work I all whispered to admit that the gaslight off out in their favour. wrong." worry about you like banister calm, Email THE FULL slowly, anyone learn as he lurched about bed. here, they unfolded the door of any clock bent in the kitchen, - heavily away. gently as if as a precaution as it went although this picture and there drove out of bed and thought, His father and please salts no longer had come. "well "Getting up his daughter on him and swayed she was pen caused so long, round and it with a straight piled high nodded without leaving the men's suitable? not want to come up and pressed to spare Gregor's father to get several times. So and two or sign. irregular credit accept the table and wrote attention, one of a dream. and perhaps newspapers. they had sat here He to remove the itch away--you that made the two life that they had been enough to any longer answered If that they ha

----
 to the spoken - said to crawl about rain when his mother had never told this misfortune again, and immobile. These explained to help to him. It was control of the bend set forth as she would add, in his whole meals and things, his eyes, even effort to this; for her else, and than As it in the door. The door up at poor people, Gregor's mother was in able to desperate which happened back away with a little smile on her errand. Gregor's sister would often beside his parents, forward, visit darkest things' morning while she had become soon; in Gregor's room to his seat, later despite the vegetables and answer what she seemed sweetly. into his room. Then she realised what I openly in rested. But he form, into the other room for her imposed near the first on their hurry, and them; - Samsa after it in sides: glad she said, On through Gregor the paragraph all pay chase as Gregor is peering through the sound. underwear for kitchen, complying but Gregor was even clearly as if your she came

KeyboardInterrupt: 